In [ ]:
from torchsummary import summary
from torchvision.datasets import ImageFolder
from torchvision.transforms.v2 import Resize, RandomRotation, Compose
from torchvision.transforms import Normalize
from torch.utils.data import DataLoader
import torch
from torch.utils.data import Dataset
from torchvision.transforms import ToTensor
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import classification_report
import torchvision.models as models
from matplotlib import pyplot as plt
import cv2
from tqdm import tqdm
import numpy as np

Загружаем датасет

In [ ]:
dataset_path = 'ogyeiv2'
train_dataset = ImageFolder(dataset_path + '/train')
val_dataset = ImageFolder(dataset_path + '/test')

In [ ]:
assert len(train_dataset.classes) == len(val_dataset.classes)
classes = train_dataset.classes
num_classes = len(classes)

for dataset in [train_dataset, val_dataset]:
    print(f"{len(dataset)} images of classes: {dataset.classes}")

Посмотрим примеры изображений

In [ ]:
fig = plt.figure(figsize=(30, 10))
for index in range(1, 3):
    image, label = train_dataset[index]
    print(classes[label])
    plt.subplot(1, 10, index)
    plt.imshow(image, cmap='gray')

Видим, что изображения имеют высокое разрешение, порядка 8Мп.

Большая часть изображения это днородный фон, который не содержит никаких полезных фич. \
Полезных метаданных (например, centercrop, в который гарантированно поместится таблетка) по этому датасету я не нашёл, поэтому начал искать примеры статей на тему распознавания таблеток.

Нашёл вот эту:
https://ietresearch.onlinelibrary.wiley.com/doi/full/10.1049/ipr2.13273

Там датасет проходил препроцессинг с помощью YOLO: вырезали ббокс с таблеткой и сохраняли только его.

Поступим так же.

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# class PreprocDataset(Dataset):
#     def __init__(self, dataset, model):
#         super(PreprocDataset, self).__init__()
#         self.dataset = dataset
#         self.model = model
#         self.processed = []

#         for x, y in tqdm(self.dataset):
#             results = self.model(x)
#             for r in results:
#                 boxes = r.boxes
#                 for box in boxes:
#                     bbox = map(int, box.xyxy.cpu().numpy())
#                     print(f"bbox: {bbox}")
#                     x = x.crop(bbox)

#                     self.processed.append(x, y)
#                     continue
#                 continue

#         self.dataset = self.processed
#         self.processed = None

#     def __len__(self):
#         return len(self.dataset)

#     def __getitem__(self, idx):
#         x, y = self.dataset[idx]
#         return x, y

In [ ]:
# from ultralytics import YOLO

# model_yolo = YOLO("yolov8m-seg.pt")
# model_yolo.to(device)

# preproc_val = PreprocDataset(val_dataset, model_yolo)
# preproc_train = PreprocDataset(train_dataset, model_yolo)

In [ ]:
# # Посмотрим на детектированные таблетки
# fig = plt.figure(figsize=(30, 10))
# for i in range(1, 4):
#     index = np.random.randint(1, 42)
#     image, label = preproc_val[index]
#     print(classes[label])
#     plt.subplot(1, 10, i)
#     plt.imshow(image, cmap='gray')

Так гораздо лучше.

Сохраним обработанный датасет на диск.

In [ ]:
# import os

# output_root_dir = "processed_ogyeiv2/train"
# output_root_dir = "processed_ogyeiv2/test"
# os.makedirs(output_root_dir, exist_ok=True)

# for i, (image, label) in enumerate(preproc_val):
#     class_name = classes[label]
#     class_dir = os.path.join(output_root_dir, class_name)
#     os.makedirs(class_dir, exist_ok=True)

#     # Save the image
#     image_filename = os.path.join(class_dir, f"image_{i:05d}.png")
#     image.save(image_filename)

Создадим датасет и загрузчик.

Вертикальный и горизонтальный поворот не используем, может "поплыть" маркировка на некоторых таблетках (aspirin, atorvastatin и пр.)

In [ ]:
train_transforms = Compose([
    RandomRotation([-5, 5], fill=255.),
    Resize((224, 224)),
    Normalize((0.5), (0.5))
])

val_transforms = Compose([
    Resize((224, 224)),
    Normalize((0.5), (0.5))
])

In [ ]:
class OgyeivDataset(Dataset):
    def __init__(self, dataset, transforms):
        super(OgyeivDataset, self).__init__()
        self.dataset = dataset
        self.transforms = transforms
        self.to_tensor = ToTensor()

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        # x копируем в vRAM, а y -нет. Почему так?
        # GPU быстрее выполнит преобразования над изображениями, поэтому их выложим в vRAM сразу.
        # Однако, y - это просто переменная типа int, лучше её скопировать в vRAM батчем.
        x, y = self.dataset[idx]
        x = self.to_tensor(x).to(device)
        return self.transforms(x), y

Используем в качестве классификатора модифицированную модель EfficientNet.


Почему именно её ? Мы хотим распознавать таблетки с точностью около 75%. Судя по изложенному в теории, это достаточно высокая точность, а значит нужна модель с большей точностью, пускай и не такая быстрая. 

Кроме того, цена ошибки распознавания достаточно высока.

Заменим полносвязный слой, т. к. нам хватит 84 классов и дообучим.

In [ ]:
model = models.efficientnet_v2_s(weights=models.EfficientNet_V2_S_Weights.IMAGENET1K_V1)

# Посмотрим на summary модели, чтобы найти какой слой надо заменить
summary(model, input_size=(3, 224, 224), device='cpu')

Замораживаем слои, подменяем последний слой, размораживаем его

In [ ]:
for param in model.parameters():
    param.requires_grad = False

model.classifier[1] = nn.Linear(
    in_features=1280,
    out_features=num_classes)

for param in model.classifier[1].parameters():
    param.requires_grad = True

summary(model, input_size=(3, 224, 224), device='cpu')

Отправляем модель на GPU для обучения

In [ ]:
model.to(device)

In [ ]:
batch_size = 64

train_dataset = OgyeivDataset(train_dataset, train_transforms)
val_dataset = OgyeivDataset(val_dataset, val_transforms)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.003)


def train_one_epoch(epoch_index):
    running_loss = 0.
    avg_loss = 0.

    for i, data in enumerate(train_loader):
        inputs, labels = data
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        avg_loss = running_loss / (i + 1)
        print(f'Эпоха: {epoch_index}, батч {i}/{len(train_loader)}, ошибка {avg_loss}')

    return avg_loss

In [ ]:
EPOCHS = 10
best_vloss = 1e5

for epoch in range(EPOCHS):
    print(f'Эпоха {epoch}')

    model.train(True)
    avg_loss = train_one_epoch(epoch)

    model.eval()
    running_vloss = 0.0

    with torch.no_grad():
        for i, vdata in enumerate(val_loader):
            vinputs, vlabels = vdata
            vlabels = vlabels.to(device)
            voutputs = model(vinputs)
            running_vloss += criterion(voutputs, vlabels)

    avg_vloss = running_vloss / (i + 1)

    if avg_vloss < best_vloss:
        best_vloss = avg_vloss
        model_path = f'pill_classifier_{epoch}.pt'
        torch.save(model.state_dict(), model_path)

    print(
        f'Конец эпохи. Ошибка обучения: {avg_loss}, ошибка валидации: {avg_vloss}')

Возвращаем модель на CPU, строим отчёт

In [ ]:
model.to('cpu')
model.eval()
labels_predicted = []
labels_true = []

with torch.no_grad():
    for data in val_loader:
        images, labels = data
        images, labels = images.to('cpu'), labels.to('cpu')

        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        labels_predicted.extend(predicted.numpy())
        labels_true.extend(labels.numpy())

print(classification_report(labels_true, labels_predicted, target_names=classes))